### Definition Generation Evaluation

In [1]:
# import gc
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import json
# model = "llama-2"
model = "llama"

# Load the JSON data
with open(f'./dataset/{model}_definition_combined.json', 'r') as json_file:
    data = json.load(json_file)

#### Dataset creation

In [4]:
# Initialize lists to store predictions and references
predictions = []
references = []
# Iterate through each item in the JSON data
for item in data:
  celex_id = item["celex_id"]
  if item["existing_record"] == ["NEW TERM"]:
    original_text = item['original_definition']
    term = item['term']
    new_term = f"'{term}'"
    original_text = original_text.replace(term, new_term)
    generated_text = item['generated_definition']

    if ((generated_text != "") and (generated_text != "NO JSON AS AN OUTPUT OBTAINED")):
      # Add the generated text to predictions list
      predictions.append(generated_text)

      # Add the original text to references list
      references_item = [original_text]  # References for this item
      references.append(references_item)
    # else:
    #   print(f"term is {term} for celex_id: {celex_id}")
    #   input("Press Enter to continue to the next iteration...")

In [5]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 7.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
  Using cached xxhash-3.4.1-cp39-cp39-macosx_11_0_arm64.whl (30 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 36.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 24.9 MB/s eta 0:00:00
  Using cached PyYAML-6.0.1-cp39-cp39-macosx_11_0_arm64.whl (174 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 4.8 MB/s eta 0:00:00
  Using cached async_ti

#### BLEU Score
- Reason for choosing
  
  - Length of the generated definitions matters
  - Precise definition is required


In [6]:
import evaluate
bleu = evaluate.load("bleu")

/Users/ashishchouhan/Desktop/PhD/6a.Production_Code/4. final_paper_code/LexDrafter/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Compute the BLEU score
bleu4_results = bleu.compute(predictions=predictions, references=references, max_order=4)
print(f"BLEU-4: {bleu4_results['bleu']}")

BLEU-4: 0.04665412304915669


In [8]:
# Compute the BLEU score
bleu3_results = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {bleu3_results['bleu']}")

BLEU-3: 0.07417244990220871


In [9]:
# Compute the BLEU score
blue2_results = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {blue2_results['bleu']}")

BLEU-2: 0.1307435422849298


In [10]:
# Compute the BLEU score
blue1_results = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {blue1_results['bleu']}")

BLEU-1: 0.250832269797197


#### BERTScore

In [11]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 8.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
import evaluate
bertscore = evaluate.load("bertscore")

In [13]:
bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

In [14]:
# Extract the F1 scores
f1_scores = bertscore_results['f1']

# Calculate the mean of F1 scores
mean_f1 = sum(f1_scores) / len(f1_scores)

In [15]:
print(f"BERTScore: {bertscore_results['f1']}")
print("--------")
print(f"Mean BERTScore: {mean_f1}")

BERTScore: [0.8096814751625061, 0.8092846870422363, 0.8128169775009155, 0.7930715084075928, 0.7977089881896973, 0.80185467004776, 0.7982388734817505, 0.7707018852233887, 0.8354070782661438, 0.8295235633850098, 0.8188246488571167, 0.7852503061294556, 0.7366086840629578, 0.8280327916145325, 0.7924872636795044, 0.8736254572868347, 0.7527464628219604, 0.7203462719917297, 0.8348162770271301, 0.7891941070556641, 0.8103252053260803, 0.879619300365448, 0.8630580306053162, 0.8443807363510132, 0.8564853668212891, 0.82341068983078, 0.8568388223648071, 0.8409414887428284, 0.8310742378234863, 0.8770844340324402, 0.7328611612319946, 0.7779026627540588, 0.7853065133094788, 0.8128845691680908, 0.839888334274292, 0.8136756420135498, 0.814499020576477, 0.851807177066803, 0.8274761438369751, 0.7964848875999451, 0.8498697280883789, 0.8367056250572205, 0.80133455991745, 0.7226757407188416, 0.7946663498878479, 0.8515051007270813, 0.8206626772880554, 0.838275134563446, 0.817543089389801, 0.8265310525894165, 

#### (NOT USED) BERTScore

In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00


In [ ]:
!pip install datasets
from datasets import load_metric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00


In [ ]:
bertscore_metric = load_metric('bertscore')

<ipython-input-3-9b26049158da>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bertscore_metric = load_metric('bertscore')


In [ ]:
def bertscore_evaluation(generated_text, actual_text):
  bert_scores = bertscore_metric.compute(predictions=[generated_text], references=[actual_text], lang="en")
  return bert_scores['f1']

In [ ]:
original_text = """
"import duty" means customs duty payable on the import of goods;"""



vicuna_text = """
Import duty is a tax or fee charged by a country on goods that are imported into that country. It is typically imposed to raise revenue for the government or to protect domestic industries from foreign competition.
"""

In [ ]:
bertscore_value = bertscore_evaluation(vicuna_text, original_text)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bertscore_value

[0.8773407340049744]

#### (NOT USED) ROUGE
- Selected: https://github.com/danieldeutsch/sacrerouge/tree/master
- Refering to the paper: https://aclanthology.org/2023.acl-long.107.pdf

In [ ]:
!pip install sacrerouge==0.2.5

In [ ]:
!pip install urllib3==1.25.10
!apt install libxml-dom-perl

In [ ]:
!sacrerouge setup-metric rouge

In [ ]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

In [ ]:
from sacrerouge.metrics import Rouge

def rouge_evaluation(generated_text, actual_text):
  rouge = Rouge(max_ngram=4)
  scores = rouge.score(generated_text, [actual_text])
  return scores

In [ ]:
original_text = """
"import duty" means customs duty payable on the import of goods;"""

In [ ]:
values = rouge_evaluation(original_text, original_text)
values